# Практическая работа №2: Обработка выборочных данных. Нахождение точечных оценок параметров распределения

Выполнил студент гр. 1384 Шаганов Вячеслав.

## Цель работы

Получение практических навыков нахождения точечных статистических оценок параметров распределения.

## Основные теоретические положения
Для статистической обработки данных применяются различные методы, позволяющие определить ключевые параметры распределения: математическое ожидание, дисперсия, стандартное отклонение, коэффициенты асимметрии и эксцесса, а также мода и медиана.

- Математическое ожидание представляет собой среднее значение случайной величины и рассчитывается по формуле:

    $E(X) = \frac{1}{n} \sum_{i=1}^{n} x_i$

    где $x_i$ — значения выборки, а $n$ — объем выборки.

- Дисперсия характеризует разброс данных и определяется как среднее квадратичное отклонение от математического ожидания:

    $D(X) = \frac{1}{n} \sum_{i=1}^{n} (x_i - E(X))^2$

- Стандартное отклонение  является квадратным корнем из дисперсии и показывает среднее отклонение значений от среднего:

    $\sigma = \sqrt{D(X)}$

- Исправленная выборочная дисперсия используется для получения более точной оценки дисперсии генеральной совокупности, учитывая, что расчеты производятся на основе выборки. Формула исправленной дисперсии учитывает уменьшение степени свободы на единицу:

    $s^2 = \frac{1}{n-1} \sum_{i=1}^{n} (x_i - \bar{x})^2 = \frac{n}{n-1} D_{sample} $

    где $ \bar{x} $ — выборочное среднее.

- Исправленное стандартное отклонение — это квадратный корень из исправленной выборочной дисперсии:

    $s = \sqrt{s^2}$


- Коэффициенты асимметрии и эксцесса измеряют форму распределения. Асимметрия показывает наклон распределения, а эксцесс — его остроту.

- Мода и медиана являются мерами центральной тенденции. Мода — наиболее часто встречающееся значение, медиана — середина упорядоченного ряда данных.

    Эти статистические меры дают представление о характере распределения и используются для его анализа и интерпретации.


## Постановка задачи

Для заданных выборочных данных вычислить с использованием метода моментов и условных вариант точечные статистические оценки математического ожидания, дисперсии, среднеквадратичного отклонения, асимметрии, эксцесса, моды, медианы и коэффициента вариации исследуемой случайной величины. Полученные результаты содержательно проинтерпретировать.

## Выполнение работы

Для начала перенесём результаты, полученные в работе №1 в данную работу.

In [4]:
import numpy as np

print_slice = lambda arr, k, prefix='': print(prefix, *arr[:k], '...', *arr[-k:], sep=' ')

with open('sample.txt', 'r') as f:
    sample = [[float(e) for e in s.split()] for s in f.readlines()]

sample = np.array(sample)
X, Y = sample[:, 0], sample[:, 1]

In [5]:
series = Y
ranked = np.sort(series)
var = np.array( [ranked[0]] + [ranked[i] for i in range(1, len(ranked)) if ranked[i] != ranked[i-1]] )
k = 1 + 3.31 * np.log10(len(series))
k = int(k) if int(k) % 2 == 1 else int(k) + 1
minval, maxval = var[[0, -1]]
h = (maxval - minval) / k

intervals = [[minval + i*h, minval + (i+1)*h] for i in range(k)]
for i in range(k):
    b = intervals[i]

bins = [np.average(b) for b in intervals] # Середины интервалов 
counts = np.array([
    len(   
        series[    # получаем элементы по фильтру
            (series < intervals[i][1])   # отбираем элементы, меньшие x_{i+1}, где [x_i, x_{i+1}) - интервал
            | ((i == k-1) & (series <= intervals[i][1]))   # учитываем, что последний интервал имеет нестрогую правую границу
        ]
    ) for i in range(k)
])

rel_counts = counts / len(series)

abs_freq = [counts[i] - (0 if i == 0 else counts[i-1]) for i in range(k)]
rel_freq = [rel_counts[i] - (0 if i == 0 else rel_counts[i-1]) for i in range(k)]

Теперь можно вычислить условные варианты. 
Константа C выбирается как середина среднего элемента интервального ряда

In [15]:
C = bins[len(bins) // 2]
u = (bins - C) / h
print("Условные варианты:", u)

Условные варианты: [-3. -2. -1.  0.  1.  2.  3.]


Элементы варианты получились целочисленными, чего и следовало ожидать.

Далее вычислим элементы сумм условных моментов:

In [55]:
nu = [rel_freq * u**k for k in range(5)]
n_u1_4 = rel_freq * (u+1)**4
# Код для генерации таблички
'''objs = [bins, rel_freq, u, nu[1], nu[2], nu[3], nu[4], n_u1_4]
for i in range(k):
    els = [round(arr[i], 5) for arr in objs]
    print(f"| {i+1}", *els,'', sep=' | ')
print(*[round(sum(arr), 6) for arr in objs], '', sep=' | ')''';

Полученные значения внесём в таблицу:
#### Таблица №1 

| $ i $ | $ x_i $  | $ n_i $  | $ u_i $  | $ n_iu_i $  | $ n_iu_i^2 $  | $ n_iu_i^3 $  | $ n_iu_i^4 $  | $ n_i(u_i + 1)^4 $  |
|-|-|-|-|-|-|-|-|-|
| 1 | 70.87857 | 0.025 | -3.0 | -0.075 | 0.225 | -0.675 | 2.025 | 0.4 | 
| 2 | 87.43571 | 0.05 | -2.0 | -0.1 | 0.2 | -0.4 | 0.8 | 0.05 | 
| 3 | 103.99286 | 0.09167 | -1.0 | -0.09167 | 0.09167 | -0.09167 | 0.09167 | 0.0 | 
| 4 | 120.55 | 0.325 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.325 | 
| 5 | 137.10714 | 0.29167 | 1.0 | 0.29167 | 0.29167 | 0.29167 | 0.29167 | 4.66667 | 
| 6 | 153.66429 | 0.14167 | 2.0 | 0.28333 | 0.56667 | 1.13333 | 2.26667 | 11.475 | 
| 7 | 170.22143 | 0.075 | 3.0 | 0.225 | 0.675 | 2.025 | 6.075 | 19.2 | 
|  $ \Sigma $  | - | 1.0 | - | 0.533333 | 2.05 | 2.283333 | 11.55 | 36.116667 | 

Проведём контроль вычислений: сумма последнего столбца должна равняться суммы предпоследних 4 столбцов, взятыми с биномиальными коэффициентами:

In [57]:
print("Следующие значения должны быть равны:", sum(n_u1_4), sum(nu[4] + 4*nu[3] + 6*nu[2] + 4*nu[1] + nu[0]))

Следующие значения должны быть равны: 36.11666666666668 36.11666666666667


Вычислим условные эмпирические моменты $ \bar{M}^*_k $ через условные варианты. С помощью условных эмпирических моментов вычислим центральные эмпирические моменты $ \bar{m}_k $.

In [83]:
Ms = [ sum(nu[i]) for i in range(len(nu)) ]
m  = [
    1, 
    0,
    (Ms[2] - Ms[1]**2)*h**2,
    (Ms[3] - 3*Ms[2]*Ms[1] + 2*Ms[1]**3)*h**3,
    (Ms[4] - 4*Ms[3]*Ms[1] + 6*Ms[2]*Ms[1]**2 - 3*Ms[1]**4)*h**4
]
# Генерация таблички
'''objs = [Ms, m]
for i in range(1, len(m)):
    els = [round(arr[i], 5) for arr in objs]
    print(f"| {i}", *els,'', sep=' | ')''';

Представим результаты в виде таблицы. Центральный момент первого порядка, очевидно, ноль.

#### Таблица №2 
|  $ k $  |  $ \bar{M}^*_k $  |  $ \bar{m}_k $  |
|----------|-------------------|-----------------|
| 1 | 0.53333 | 0 | 
| 2 | 2.05 | 484.0076 | 
| 3 | 2.28333 | -3146.67483 | 
| 4 | 11.55 | 746624.1059 | 

Вычислим выборочные среднее $ \bar x_{\textit{в}} $ и дисперсию $ D_{\textit{в}} $ с помощью стандартной формулы и с помощью условных вариант. Вычислим выборочное СКО $ \sigma_{\textit{в}} $.

In [95]:
mean = np.mean(series)
disp = np.var(series)

interval_mean = np.average(bins, weights=rel_freq)
interval_disp = np.average((bins - interval_mean) ** 2, weights=rel_freq)

u_mean = Ms[1] * h + C
u_disp = m[2]

print("Выборочные средние:", mean, '|', u_mean, interval_mean)
print("Выборочные дисперсии:", disp, '|', u_disp, interval_disp)

print("Выборочное СКО:", np.sqrt(disp), '|', np.sqrt(u_disp), np.sqrt(interval_disp))

Выборочные средние: 129.40416666666667 | 129.38047619047617 129.3804761904762
Выборочные дисперсии: 481.13923263888887 | 484.0075984126984 484.0075984126984
Выборочное СКО: 21.934886200728027 | 22.000172690519918 22.000172690519918


Как можно видеть, выборочные дисперсия и среднее полностью совпадают при вычислении с помощью условных вариант и при их вычислении для интервального ряда с помощью стандартных формул. Также видно, что данные значения отличаются от выборочных значений данных параметров при их вычислении на исходной выборке, но относительные отличия менее 1%, что позволяет работать со значениями, полученными при помощи условных вариант без сильной потери точности. 

Вычислим исправленную выборочную дисперсию $ s^2 $ и исправленное СКО $ s $. Сравним данные оценки со смещёнными оценками дисперсии и СКО.

In [ ]:
s2 = len(series) / (len(series)-1) * u_disp
s = np.sqrt(s2)
print(f"Исправленная выборочная дисперсия: {s2}\nИсправленное СКО: {s}")

Исправленная выборочная дисперсия: 488.0748891556623
Исправленное СКО: 22.092417005743446


Видим, что данные значения чуть больше смещённых значений тех же параметров. 

Найдём статистическую оценку коэффициентов асимметрии $ \bar{A}_s $ и эксцесса $ \bar{E} $. 

In [101]:
As = m[3] / s**3
E = m[4] / s**4 - 3
print(f"Стат. оценка коэффициента асимметрии: {As}\nСтат. оценка коэффициента эксцесса {E}")

Стат. оценка коэффициента асимметрии: -0.2918247822547876
Стат. оценка коэффициента эксцесса 0.13421712369086736


Отрицательное значение коэффициента асимметрии говорит о том, что распределение смещено в правую сторону, то есть мода находится правее среднего значения. 

Положительное значение коэффициента эксцесса говорит о том, что пик распределения около мат. ожидания острый (острее, чем у нормального распределения)

Однако малые значения данных коэффициентов говорят о небольшом отличии от нормального распределения.

Для интервального ряда вычислим моду $ M^*_o $, медиану $ M^*_e $ и коэффициент вариации $ V^* $ заданного распределения.

In [136]:
# Mode 
i = np.argmax(abs_freq)
Mo = bins[i-1] + h * (abs_freq[i] - abs_freq[i-1]) / ((abs_freq[i] - abs_freq[i-1]) + (abs_freq[i] - abs_freq[i+1]))
print("Мода:", Mo)

# Median
i = 0
for j in range(len(counts)):
    if rel_counts[j] >= 0.5:
        i = j
        break
Me = bins[i-1] + h / abs_freq[i] * (0.5 - sum([h/abs_freq[j] for j in range(i)]))
print("Медиана:", Me)

# Var coef
sigma = np.sqrt(disp)

V = sigma / mean 
print(f"Коэффициент вариации: {V*100:.2f}%")

Мода: 118.48035714285714
Медиана: 115.95737576029413
Коэффициент вариации: 16.95%


Что мода, что медиана, находятся внутри самого частотного интервала. Как и говорилось ранее, мода оказалась правее медианы. Коэффициент вариации говорит о несильной изменчивости по отношению к среднему значению.

Далее проведем аналогичные вычисления для выборки X 

In [146]:
series = X
ranked = np.sort(series)
var = np.array( [ranked[0]] + [ranked[i] for i in range(1, len(ranked)) if ranked[i] != ranked[i-1]] )
k = 1 + 3.31 * np.log10(len(series))
k = int(k) if int(k) % 2 == 1 else int(k) + 1
minval, maxval = var[[0, -1]]
h = (maxval - minval) / k

intervals = [[minval + i*h, minval + (i+1)*h] for i in range(k)]
for i in range(k):
    b = intervals[i]

bins = [np.average(b) for b in intervals] # Середины интервалов 
counts = np.array([
    len(   
        series[    # получаем элементы по фильтру
            (series < intervals[i][1])   # отбираем элементы, меньшие x_{i+1}, где [x_i, x_{i+1}) - интервал
            | ((i == k-1) & (series <= intervals[i][1]))   # учитываем, что последний интервал имеет нестрогую правую границу
        ]
    ) for i in range(k)
])

rel_counts = counts / len(series)

abs_freq = [counts[i] - (0 if i == 0 else counts[i-1]) for i in range(k)]
rel_freq = [rel_counts[i] - (0 if i == 0 else rel_counts[i-1]) for i in range(k)]

C = bins[len(bins) // 2]
u = (bins - C) / h
print("Условные варианты:", u)

nu = [rel_freq * u**k for k in range(5)]
n_u1_4 = rel_freq * (u+1)**4

print("Следующие значения должны быть равны:", sum(n_u1_4), sum(nu[4] + 4*nu[3] + 6*nu[2] + 4*nu[1] + nu[0]))

Ms = [ sum(nu[i]) for i in range(len(nu)) ]
m  = [
    1, 
    0,
    (Ms[2] - Ms[1]**2)*h**2,
    (Ms[3] - 3*Ms[2]*Ms[1] + 2*Ms[1]**3)*h**3,
    (Ms[4] - 4*Ms[3]*Ms[1] + 6*Ms[2]*Ms[1]**2 - 3*Ms[1]**4)*h**4
]

mean = np.mean(series)
disp = np.var(series)

interval_mean = np.average(bins, weights=rel_freq)
interval_disp = np.average((bins - interval_mean) ** 2, weights=rel_freq)

u_mean = Ms[1] * h + C
u_disp = m[2]

print("Выборочные средние:", mean, '|', u_mean, interval_mean)
print("Выборочные дисперсии:", disp, '|', u_disp, interval_disp)

print("Выборочное СКО:", np.sqrt(disp), '|', np.sqrt(u_disp), np.sqrt(interval_disp))

s2 = len(series) / (len(series)-1) * u_disp
s = np.sqrt(s2)
print(f"Исправленная выборочная дисперсия: {s2}\nИсправленное СКО: {s}")

As = m[3] / s**3
E = m[4] / s**4 - 3
print(f"Стат. оценка коэффициента асимметрии: {As}\nСтат. оценка коэффициента эксцесса {E}")

Условные варианты: [-3. -2. -1.  0.  1.  2.  3.]
Следующие значения должны быть равны: 21.00833333333334 21.008333333333344
Выборочные средние: 452.2583333333333 | 449.89285714285717 449.89285714285717
Выборочные дисперсии: 3074.3582638888893 | 2975.406887755102 2975.406887755102
Выборочное СКО: 55.44689589047244 | 54.54729037958808 54.54729037958808
Исправленная выборочная дисперсия: 3000.410306979935
Исправленное СКО: 54.776001195596
Стат. оценка коэффициента асимметрии: 0.12067122312284088
Стат. оценка коэффициента эксцесса -0.33419579993017834


Полученные результаты занесём в таблицу:

#### Таблица №3

| $ i $ | $ x_i $  | $ n_i $  | $ u_i $  | $ n_iu_i $  | $ n_iu_i^2 $  | $ n_iu_i^3 $  | $ n_iu_i^4 $  | $ n_i(u_i + 1)^4 $  |
|-|-|-|-|-|-|-|-|-|
| 1 | 340.28571 | 0.05 | -3.0 | -0.15 | 0.45 | -1.35 | 4.05 | 0.8 | 
| 2 | 378.85714 | 0.11667 | -2.0 | -0.23333 | 0.46667 | -0.93333 | 1.86667 | 0.11667 | 
| 3 | 417.42857 | 0.24167 | -1.0 | -0.24167 | 0.24167 | -0.24167 | 0.24167 | 0.0 | 
| 4 | 456.0 | 0.28333 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.28333 | 
| 5 | 494.57143 | 0.19167 | 1.0 | 0.19167 | 0.19167 | 0.19167 | 0.19167 | 3.06667 | 
| 6 | 533.14286 | 0.075 | 2.0 | 0.15 | 0.3 | 0.6 | 1.2 | 6.075 | 
| 7 | 571.71429 | 0.04167 | 3.0 | 0.125 | 0.375 | 1.125 | 3.375 | 10.66667 | 
|  $ \Sigma $   | - | 1.0 | - | -0.158333 | 2.025 | -0.608333 | 10.925 | 21.008333 | 

#### Таблица №4
|  $ k $  |  $ \bar{M}^*_k $  |  $ \bar{m}_k $  |
|----------|-------------------|-----------------|
| 1 | -0.15833 | 0 | 
| 2 | 2.025 | 2975.40689 | 
| 3 | -0.60833 | 19832.37327 | 
| 4 | 10.925 | 23998801.03784 | 

Для интервального ряда вычислим моду $ M^*_o $, медиану $ M^*_e $ и коэффициент вариации $ V^* $ заданного распределения.

In [147]:
# Mode 
i = np.argmax(abs_freq)
Mo = bins[i-1] + h * (abs_freq[i] - abs_freq[i-1]) / ((abs_freq[i] - abs_freq[i-1]) + (abs_freq[i] - abs_freq[i+1]))
print("Мода:", Mo)

# Median
i = 0
for j in range(len(counts)):
    if rel_counts[j] >= 0.5:
        i = j
        break
Me = bins[i-1] + h / abs_freq[i] * (0.5 - sum([h/abs_freq[j] for j in range(i)]))
print("Медиана:", Me)

# Var coef
sigma = np.sqrt(disp)

V = sigma / mean 
print(f"Коэффициент вариации: {V*100:.2f}%")

Мода: 429.4821428571429
Медиана: 406.0684658099693
Коэффициент вариации: 12.26%


Так как коэффициент вариации данной выборки меньше, чем у выборки Y, можно говорить о том, что данная выборка обладает меньшей изменчивостью относительно среднего значения.

## Выводы

В ходе выполнения работы были успешно рассчитаны точечные статистические оценки для заданных выборочных данных, включая математическое ожидание, дисперсию и стандартное отклонение. Результаты показали согласованность между различными методами, что подтверждает их корректность.

Анализ коэффициентов вариации, асимметрии и эксцесса выявил характерные особенности распределения. 